In [2]:
#from gurobipy import*
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math


K = set([1])                        #K=消防車集合
N = set([1, 2, 3, 4, 5])            #所有點的集合
N_D = set([4])                      #Depot
N_F = set([5])                      #起火點
Q={1:6,2:2,3:4,5:6}                  #q=單位時間火在i點燒的燃料量
b={1:18,2:8,3:12,5:18}               #b=節點i的價值
P=3                                  #1號消防車單位時間處理的燃料量
H={1:2,2:3,3:3,5:2}                  #h=火燒i點需要的時間
T_number = 40                        #時間長度
T = list([i for i in range(T_number+1)])    #時間集合
M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}

df = pd.read_excel("firefighter_route.xlsx")
df_num = len(df.index)
for i in N:
    A_p.append((i,i))
for i in range(0,df_num):
    u = df.iloc[i]['i']
    v = df.iloc[i]['j']
    fighterIndex = df.iloc[i]['k']
    time = df.iloc[i]['travel time']
    A_p.append((u,v))
    tau[u,v,fighterIndex] = time
df = pd.read_excel("fire_route.xlsx")
df_num = len(df.index)
for i in range(0,df_num):
    u = df.iloc[i]['i']
    v = df.iloc[i]['j']
    time = df.iloc[i]['travel time']
    A_f.append((u,v))
    lamb[u,v] = time
for i in Q:
    process[i] = math.ceil(Q[i]*H[i]/P)


A_f_NEIGHBOR={}                      #A_f_NEIGHBOR=與點i相鄰的點

for l in N-N_D:                          #定義A_f_NEIGHBOR
    A_f_NEIGHBOR[l]=[]
    for i in range(len(A_f)):
        if A_f[i][1]==l and A_f[i][0]!=l:
            A_f_NEIGHBOR[l].append(A_f[i][0])
            
#     connect = A_f.select('*',l)
#     A_f_NEIGHBOR[l]=[]
#     for temp in connect:
#         A_f_NEIGHBOR[l].append(temp[0])
        



A_f_NEIGHBOR_T={}                    #A_f_NEIGHBOR_T=紀錄 t-hi-Lambda(i,j)>=0 且 與j點相鄰的i點

for j in N-N_D-N_F:                         #定義A_f_NEIGHBOR_T
    for t in T:
        A_f_NEIGHBOR_T[j,t]=[]
        for i in A_f_NEIGHBOR[j]:
            if t-H[i]-lamb[i,j]>=0:
                A_f_NEIGHBOR_T[j,t].append(i)

model=gp.Model("FIREFIGHTER")


x={}
w={}
u={}
u_bar={}
v={}
v_bar={}

#定義x[i,j,k,t]
for k in K:
    for t in T:
        for i in range(len(A_p)):
                x[A_p[i][0],A_p[i][1],k,t]=model.addVar(vtype='B',name="x[%d,%d,%d,%d]" %(A_p[i][0],A_p[i][1],k,t))

#定義w[i,k,t]
for k in K:
    for t in T:
        for i in N:
            w[i,k,t]=model.addVar(vtype='B',name="w[%d,%d,%d]" %(i,k,t))

#定義u[i,t]
u = model.addVars(N, T, vtype="B")

#定義u_bar[i,k,t]
u_bar = model.addVars(N, K, T, vtype="B", name="u_bar")

#定義v[i,t]
v = model.addVars(N, T, vtype="B", name="v")

#定義v_bar[i,t]
v_bar = model.addVars(N, T, vtype="B", name="v_bar")

model.update()

for k in K:                        #原點flow blance
    model.addConstr(gp.quicksum(x[i,j,k,0] for i,j in A_p) <= 1)

#depart from depot
for O in N_D:
    connect = A_p.select(O,'*')
    for k in K:
        model.addConstr(gp.quicksum(x[i,j,k,0] for i, j in connect) == 1)
#depot can't be processed
model.addConstrs((u_bar[i,k,t]==0 for i in N_D for k in K for t in range(T_number+1)))

#flow balance
for t in range(1,T_number):
    for k in K:
        for j in N:
            in_connect = A_p.select('*',j)
            out_connect = A_p.select(j,'*')
            temp=0 #in-degree
            temp += w[j,k,t-1] # t-1在j idle
            if j in N_D: #j in depot set會有u_bar，只是都為0
                temp+=u_bar[j,k,t]
            else: #j not in depot set若現在的t>process time，則會有u_bar
                if process[j] <= t:
                    temp += u_bar[j,k,t-process[j]]
            for m,n in in_connect:
                if m != n and tau[m,n,k] <= t: #若現在的t>travel time，則會有x
                    temp += x[m,n,k,t-tau[m,n,k]]
            model.addConstr(temp == gp.quicksum(x[n,w,k,t] for n,w in out_connect)) #in-degree = out-degree
model.addConstrs((u_bar[i,k,t] + w[i,k,t] == x[i,i,k,t] for i in N-N_D for k in K for t in range(T_number)))
model.addConstrs((w[s,k,t] == x[s,s,k,t]) for s in N_D for k in K for t in range(T_number))

#######################################################################################################################
model.addConstrs(u[i, t]+u_bar.sum(i, '*', t) <= 1 for i in N for t in T) #constrain 6
model.addConstrs(v[i, t] + u[i, t] == v[i, t+1] for i in N for t in T[0:-1]) #constrain 7
model.addConstrs(v_bar[i, t] + u_bar.sum(i, '*', t) == v_bar[i, t+1] for i in N for t in T[0:-1]) #constrain 9
model.addConstrs(u_bar.sum(s, '*', '*') == 0 for s in N_D) #constrain 11
model.addConstrs(u.sum(s, '*') == 0 for s in N_D) #constrain 12

#######################################################################################################################

for j in N-N_D-N_F:                            #火焰的延燒
    for t in range(T_number):
        if len(A_f_NEIGHBOR_T[j,t])==0:
            model.addConstr(u[j,t] == 0)
        else:
            model.addConstr(gp.quicksum(u[i,t-H[i]-lamb[i,j]] for i in A_f_NEIGHBOR_T[j,t]) / M <= u[j,t] + v[j,t] + v_bar[j,t+1])
            model.addConstr(gp.quicksum(u[i,t-H[i]-lamb[i,j]] for i in A_f_NEIGHBOR_T[j,t]) >= u[j,t])

for i in N_F:                           #起火點起火
    model.addConstr(u[i,0]==1)

for i in N:                            #所有節點不會在t-1~t已經被燒
    model.addConstr(v[i,0]==0)

for i in N:                            #所有節點不會在t-1~t已經被澆水
    model.addConstr(v_bar[i,0]==0)

for k in K:                             #消防員不能去已經被燃燒的節點
    for t in T:
        for l in range(len(A_p)):
            if A_p[l][1] not in N_D:
                if A_p[l][0] ==  A_p[l][1]:
                    if t+2 <= T_number:
                        model.addConstr(M*(1-v[A_p[l][1],t+2]) >= x[A_p[l][0], A_p[l][1],k,t])
                    else:
                        model.addConstr(M*(1-v[A_p[l][1],T_number]) >= x[A_p[l][0], A_p[l][1],k,t])
                elif t + tau[A_p[l][0],A_p[l][1],k] + 1 <= T_number:
                    model.addConstr(M*(1-v[A_p[l][1],t + tau[A_p[l][0],A_p[l][1],k] + 1]) >= x[A_p[l][0],A_p[l][1],k,t])
                else:
                    model.addConstr(M*(1-v[A_p[l][1],T_number]) >= x[A_p[l][0],A_p[l][1],k,t])


model.setObjective(gp.quicksum(gp.quicksum(u[i,t] for t in T) * b[i] for i in N-N_D),GRB.MINIMIZE)


model.optimize()
model.write('test.lp')
print("optimal value : ",model.ObjVal)


Set parameter Username
Academic license - for non-commercial use only - expires 2024-07-23
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i5-13500, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 1897 rows, 1804 columns and 6172 nonzeros
Model fingerprint: 0x0d13cbd5
Variable types: 0 continuous, 1804 integer (1804 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [8e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 1489 rows and 1397 columns
Presolve time: 0.02s
Presolved: 408 rows, 407 columns, 1355 nonzeros
Variable types: 0 continuous, 407 integer (407 binary)
Found heuristic solution: objective 26.0000000

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.03 work units)
Thread count was 20 (of 20 available processors)

Solution count 1: 26 

Optimal solution 

In [3]:
EPS=1.e-2

print("x:")
for (k,t,i,j) in x:
    if x[k,t,i,j].X >EPS:
        print("x[%d,%d,%d,%d]" %(k,t,i,j) , x[k,t,i,j].X)

print("w:")
for (k,t,i) in w:
    if w[k,t,i].X>EPS:
        print("w[%d,%d,%d]" %(k,t,i) , w[k,t,i].X)

print("u:")
for (t,i) in u:
    if u[t,i].X>EPS:
        print("u[%d,%d]" %(t,i) , u[t,i].X)

print("u_bar:")
for (k,t,i) in u_bar:
    if u_bar[k,t,i].X>EPS:
        print("u_bar[%d,%d,%d]" %(k,t,i) , u_bar[k,t,i].X)

print("v:")
for (t,i) in v:
    if v[t,i].X>EPS:
        print("v[%d,%d]" %(t,i) , v[t,i].X)

print("v_bar:")
for (t,i) in v_bar:
    if v_bar[t,i].X>EPS:
        print("v_bar[%d,%d]" %(t,i) , v_bar[t,i].X)




x:
x[4,3,1,0] 1.0
x[3,3,1,2] 1.0
x[3,1,1,6] 1.0
x[1,1,1,10] 1.0
x[1,4,1,14] 1.0
x[4,4,1,18] 1.0
x[4,4,1,19] 1.0
x[4,4,1,20] 1.0
x[4,4,1,21] 1.0
x[4,4,1,22] 1.0
x[4,4,1,23] 1.0
x[4,4,1,24] 1.0
x[4,4,1,25] 1.0
x[4,4,1,26] 1.0
x[4,4,1,27] 1.0
x[4,4,1,28] 1.0
x[4,4,1,29] 1.0
x[4,4,1,30] 1.0
x[4,4,1,31] 1.0
x[4,4,1,32] 1.0
x[4,4,1,33] 1.0
x[4,4,1,34] 1.0
x[4,4,1,35] 1.0
x[4,4,1,36] 1.0
x[4,1,1,37] 1.0
w:
w[4,1,18] 1.0
w[4,1,19] 1.0
w[4,1,20] 1.0
w[4,1,21] 1.0
w[4,1,22] 1.0
w[4,1,23] 1.0
w[4,1,24] 1.0
w[4,1,25] 1.0
w[4,1,26] 1.0
w[4,1,27] 1.0
w[4,1,28] 1.0
w[4,1,29] 1.0
w[4,1,30] 1.0
w[4,1,31] 1.0
w[4,1,32] 1.0
w[4,1,33] 1.0
w[4,1,34] 1.0
w[4,1,35] 1.0
w[4,1,36] 1.0
u:
u[2,7] 1.0
u[5,0] 1.0
u_bar:
u_bar[1,1,10] 1.0
u_bar[3,1,2] 1.0
v:
v[2,8] 1.0
v[2,9] 1.0
v[2,10] 1.0
v[2,11] 1.0
v[2,12] 1.0
v[2,13] 1.0
v[2,14] 1.0
v[2,15] 1.0
v[2,16] 1.0
v[2,17] 1.0
v[2,18] 1.0
v[2,19] 1.0
v[2,20] 1.0
v[2,21] 1.0
v[2,22] 1.0
v[2,23] 1.0
v[2,24] 1.0
v[2,25] 1.0
v[2,26] 1.0
v[2,27] 1.0
v[2,28] 1.0
v[2,29] 1.0

In [8]:
import json
data={}

NODE_POS = {1:(30, 300), 2:(100, 100), 3:(500, 250), 4:(450, 600), 5:(100, 150)}
data['NODE_POS'] = NODE_POS
data['N'] = list(N)
data['N_D'] = list(N_D)
data['N_F'] = list(N_F)
data['K'] = list(K)

temp = {}
for (i, j, k, t) in x:
    temp[str((i, j, k, t))] = x[i, j, k, t].X
data['x'] = temp

temp = {}
for (k, t, i) in w:
    temp[str((k, t, i))] = w[k, t, i].X
data['w'] = temp

temp = {}
for (t, i) in u:
    temp[str((t, i))] = u[t, i].X
data['u'] = temp

temp = {}
for (t, i, k) in u_bar:
    temp[str((t, i, k))] = u_bar[t, i, k].X
data['u_bar'] = temp

temp = {}
for (t, i) in v:
    temp[str((t, i))] = v[t, i].X
data['v']  = temp

temp = {}
for (t, i) in v_bar:
    temp[str((t, i))] = v_bar[t, i].X
data['v_bar'] = temp
json_data = json.dumps(data)
with open("data.json", "w") as file:
    file.write(json.dumps(data))

In [18]:
raw_data = json.loads(json_data)
new_x = {}
for i in raw_data['x']:
    new_x[tuple(map(int, i[1:-1].split(',')))] = raw_data['x'][i]
    
print(new_x)

{(1, 1, 1, 0): 0.0, (2, 2, 1, 0): 0.0, (3, 3, 1, 0): 0.0, (4, 4, 1, 0): 0.0, (5, 5, 1, 0): 0.0, (1, 2, 1, 0): 0.0, (2, 1, 1, 0): 0.0, (1, 3, 1, 0): 0.0, (3, 1, 1, 0): 0.0, (1, 4, 1, 0): 0.0, (4, 1, 1, 0): 0.0, (2, 5, 1, 0): 0.0, (5, 2, 1, 0): 0.0, (3, 4, 1, 0): 0.0, (4, 3, 1, 0): 1.0, (3, 5, 1, 0): 0.0, (5, 3, 1, 0): 0.0, (4, 5, 1, 0): 0.0, (5, 4, 1, 0): 0.0, (1, 1, 1, 1): 0.0, (2, 2, 1, 1): 0.0, (3, 3, 1, 1): 0.0, (4, 4, 1, 1): 0.0, (5, 5, 1, 1): 0.0, (1, 2, 1, 1): 0.0, (2, 1, 1, 1): 0.0, (1, 3, 1, 1): 0.0, (3, 1, 1, 1): 0.0, (1, 4, 1, 1): 0.0, (4, 1, 1, 1): -0.0, (2, 5, 1, 1): 0.0, (5, 2, 1, 1): 0.0, (3, 4, 1, 1): 0.0, (4, 3, 1, 1): 0.0, (3, 5, 1, 1): 0.0, (5, 3, 1, 1): 0.0, (4, 5, 1, 1): 0.0, (5, 4, 1, 1): 0.0, (1, 1, 1, 2): 0.0, (2, 2, 1, 2): 0.0, (3, 3, 1, 2): 1.0, (4, 4, 1, 2): 0.0, (5, 5, 1, 2): 0.0, (1, 2, 1, 2): 0.0, (2, 1, 1, 2): 0.0, (1, 3, 1, 2): 0.0, (3, 1, 1, 2): 0.0, (1, 4, 1, 2): 0.0, (4, 1, 1, 2): -0.0, (2, 5, 1, 2): 0.0, (5, 2, 1, 2): 0.0, (3, 4, 1, 2): 0.0, (4, 3, 1,